# Set up

In [3]:
import os
import pandas as pd
pd.set_option("display.max_colwidth", None)

os.environ["AZURE_OPENAI_ENDPOINT"] = "https://dalle3-swo.openai.azure.com/"
os.environ["AZURE_OPENAI_API_KEY"] = "e51119f8d8774069a6594d92ccf7a70d"

In [2]:
from langchain_community.utilities import GoogleSearchAPIWrapper
from langchain_community.vectorstores import Chroma
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings, OpenAIEmbeddings
from langchain_core.output_parsers import BaseOutputParser
from typing import List, Optional
import re
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import chain
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_core.output_parsers import StrOutputParser

# LLM
llm = AzureChatOpenAI(
    openai_api_version="2024-02-15-preview",
    azure_deployment="gpt-35-turbo",
    temperature=0
) 

gpt_35_turbo_16k = AzureChatOpenAI(
    openai_api_version="2024-02-15-preview",
    azure_deployment="gpt-35-turbo-16k",
    temperature=0
) 

smart_llm = AzureChatOpenAI(
    openai_api_version="2024-02-15-preview",
    azure_deployment="gpt-4",
    temperature=0
) 
gpt_35_turbo_instruct = AzureChatOpenAI(
    openai_api_version="2024-02-15-preview",
    azure_deployment="gpt-35-turbo-instruct",
    temperature=0
) 
# Embedding
embeddings = AzureOpenAIEmbeddings(
    azure_deployment="text-embedding-ada-002",
    openai_api_version="2024-02-15-preview",
)

embeddings_3_large = AzureOpenAIEmbeddings(
    azure_deployment="text-embedding-3-large",
    openai_api_version="2024-02-15-preview",
)

# Text loader

In [38]:
from langchain_community.document_loaders import AzureAIDocumentIntelligenceLoader

endpoint = "https://di-buv.cognitiveservices.azure.com/"
key = "fe9f945227944764889fd95f3bb2abda"



loader = AzureAIDocumentIntelligenceLoader(
            api_endpoint=endpoint, 
            api_key=key, 
            file_path="../../data/qna_update_data/SU-JUL24-FAQ.docx", 
            api_model="prebuilt-layout"
)
azure_ai_DI_data = loader.load()

In [39]:
print(azure_ai_DI_data[0].page_content)

<!-- PageHeader="STUDENT ENGAGEMENT" -->

<!-- PageHeader="STUDENT INFORMATION OFFICE" -->

STUDENTS AND PARENTS’ FREQUENTLY ASKED QUESTIONS

CÁC CÂU HỎI THƯỜNG GẶP DÀNH CHO SINH VIÊN VÀ PHỤ HUYNH

CONTENTS/MỤC LỤC

I. ACADEMIC AND STUDENT OPERATIONS 12

1\. How can students/parents get a timetable? / Sinh viên/phụ huynh có thể nhận thời khóa biểu ở đâu? 12

2\. How does BUV record student attendance? / Điểm chuyên cần của sinh viên được BUV quản lý như thế nào? 12

3\. What should students/parents do if students need to be absent from classes? / Nếu sinh viên cần nghỉ học vì một lý do nào đó, sinh viên/phụ huynh cần phải làm gì? 16

4\. If the academic results of students are not good enough and students/parents are worried, what can BUV support to improve their performance? / Sinh viên/phụ huynh lo rằng kết quả học tập của sinh viên không đủ tốt, liệu nhà trường có kế hoạch gì trong việc giúp đỡ sinh viên cải thiện kết quả học tập không? 20

5\. In the students’ timetable, I have see

# Splitter

Note: the splitter split the q&a based on **\n\nAnswer / Câu trả lời:\n\n**, so if there're no those words between question and answer, the split would miss that question --> need to verify the number of question == len(document) or not

In [40]:
import re  
  
def extract_qna_from_extracted_word(extracted_word):
    raw_splitted_text = re.split(r"\n\nAnswer / Câu trả lời:\n\n", extracted_word)
    q_a_list = []
    for element in raw_splitted_text:
        split_question = re.split(r"\n\n", element)
        if len(split_question) == 1:
            current_quesions = split_question[0]
            q_a_list.append(current_quesions)
        else:
            answer_of_current_quesions, next_question = split_question[:-1], split_question[-1]
            answer_of_current_quesions = "\n\n".join(answer_of_current_quesions)
            q_a_list.extend([answer_of_current_quesions, next_question])
    return q_a_list

In [41]:
list_qna = extract_qna_from_extracted_word(azure_ai_DI_data[0].page_content)
refine_qna_list = list_qna[1:-1]

In [42]:
from langchain.docstore.document import Document

buv_docs = []
for i in range(int(len(refine_qna_list)/2)):
    # page_content is question, metadata is answer
    document = Document(page_content=refine_qna_list[2*i] + "\n\nAnswer / Câu trả lời:\n\n" + refine_qna_list[2*i+1])
    buv_docs.append(document)

In [43]:
len(buv_docs)

83

In [47]:
print(buv_docs[65].page_content)

My Exceptional Circumstance claim got upheld. What is the next step? / Sinh viên cần làm gì sau khi đơn xin xem xét Trường hợp Ngoại lệ được chấp thuận?

Answer / Câu trả lời:

If the student gets Exceptional Circumstance upheld, one of the following outcomes can be applied:

For Upheld Exceptional Circumstances claims:

If you have submitted your work within one week of the deadline, it will be marked and counted as if it had been submitted on time. A successful Exceptional Circumstances claim can only extend your assessment hand in date by one week and only for assessments where this applies. For example, you cannot be offered an extension for an Exam that you were unable to sit. If you are unsure whether your assessment falls into this category, please speak to your course team.

If you were unable to submit your work, or you have received a failed mark, you won’t be penalised if you are given another assessment opportunity. For example;

If you have ECs for your first assessment op

In [48]:
buv_qna = pd.DataFrame({
    "page_content": [doc.page_content for doc in buv_docs],
    "file_name": 83*['SU-JUL24-FAQ.pdf'],
    "page": 83*[0]
})

In [49]:
buv_qna.to_excel('su_jul24_qna.xlsx', index=False)

In [15]:
buv_qna_df = pd.read_excel('su_jul24_qna.xlsx')